In [1]:
import sys
import pandas as pd
sys.path.insert(0, '..')
from src.eventstream import Eventstream, EventstreamSchema, RawDataSchema
from src.tooling.step_matrix import StepMatrix
import pytest
import os

/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable filter was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable negative_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable positive_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()


ПРИ ПРОГОНЕ ТУТОРИАЛА НЕ ОТОБРАЖАЛАСЬ СТРОЧКА ENDED - И НЕ ДОЛЖНА

    1. Количество шагов у каждого пользователя в датасете [2, 100]
    2. Максимальное количество шагов в матрице [0, 2, 16, 50, 100, 200]
    3. Дробное число шагов? Отрицательное? ("ValueError: max() arg is an empty sequence"). Ноль это тоже отрицательный тест!
2) Пороги
    1. Отрицательные, целые, ноль, один и т.д. [-50, -0.02, 0, 0.1, 0.05, 1, 67, 1000]
    2. При прогоне туториала в 
3)

In [2]:
# базовый датасет в формате датафрейма
raw_data = pd.read_csv("../src/datasets/data/simple-onlineshop.csv")

In [3]:
#пользователи отсортированные по количеству cобытий
raw_data.groupby(by='user_id', as_index=False).count().sort_values(by=['event'])

,user_id,event,timestamp
3750,999941967,2,2
447,132638296,2,2
1927,517179956,2,2
1941,519785837,2,2
1944,521074614,2,2
...,...,...,...
2881,760510988,83,83
265,78179456,83,83
3377,898482504,92,92
3621,962952209,94,94


In [4]:
min_table = raw_data.loc[raw_data['user_id'] == 999941967] #пользователь с двумя событиями

In [5]:
minmax_table = min_table.append(raw_data.loc[raw_data['user_id'] == 530349536], ignore_index = True) # минимум и максимум вместе 
minmax_table

,user_id,event,timestamp
0,999941967,catalog,2020-04-17 12:18:49.391199
1,999941967,lost,2020-04-17 12:18:50.391199
2,530349536,catalog,2019-11-12 10:03:06.441295
3,530349536,cart,2019-11-12 10:03:34.490371
4,530349536,delivery_choice,2019-11-12 10:03:36.538909
...,...,...,...
97,530349536,main,2020-03-15 03:08:17.586834
98,530349536,catalog,2020-03-15 03:08:18.914158
99,530349536,cart,2020-03-15 03:08:23.925717
100,530349536,delivery_choice,2020-03-15 03:08:27.382559


In [6]:
two_users_table = minmax_table[:8] # 2 пользователя, максимум 6 шагов

In [7]:
# create data schema
raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

# create source eventstream
two_users_table_es = Eventstream(
    raw_data=two_users_table,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

## Max steps

При проведении тестов логичнее пользоваться экземпляром класса StepMatrix, т.к. если записывать его как метод eventstream, он автоматически вызывает функцию plot(), которая мешает интерпретации результатов

In [15]:
#max step == 6
step_matrix = StepMatrix(eventstream=two_users_table_es, 
                          max_steps=6)
result = step_matrix._get_plot_data()[0]
display(result)
result.columns

,1,2,3,4,5,6
catalog,1.0,0.0,0.0,0.0,0.0,0.0
cart,0.0,0.5,0.0,0.0,0.0,0.0
delivery_choice,0.0,0.0,0.5,0.0,0.0,0.0
delivery_courier,0.0,0.0,0.0,0.5,0.0,0.0
payment_choice,0.0,0.0,0.0,0.0,0.5,0.0
payment_card,0.0,0.0,0.0,0.0,0.0,0.5
lost,0.0,0.5,0.0,0.0,0.0,0.0


Int64Index([1, 2, 3, 4, 5, 6], dtype='int64')

In [9]:
correct_result = pd.DataFrame(
            [
                [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.5, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.5, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.5, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.5, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.5],
                [0.0, 0.5, 0.0, 0.0, 0.0, 0.0]
                
            ], index=['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'lost'], columns=[1, 2 , 3, 4, 5, 6],
        )
correct_result

,1,2,3,4,5,6
catalog,1.0,0.0,0.0,0.0,0.0,0.0
cart,0.0,0.5,0.0,0.0,0.0,0.0
delivery_choice,0.0,0.0,0.5,0.0,0.0,0.0
delivery_courier,0.0,0.0,0.0,0.5,0.0,0.0
payment_choice,0.0,0.0,0.0,0.0,0.5,0.0
payment_card,0.0,0.0,0.0,0.0,0.0,0.5
lost,0.0,0.5,0.0,0.0,0.0,0.0


In [16]:
#assert
def compare_results(res, corr_res):
    return res.compare(corr_res).shape == (0, 0)
compare_results(result, correct_result)

ValueError: Can only compare identically-labeled DataFrame objects

In [11]:
#max step == 1
correct_result = pd.DataFrame(
            [
                [1.0]
            ], index=['catalog'], columns=[1],
        )
correct_result

,1
catalog,1.0


In [18]:
step_matrix = StepMatrix(eventstream=two_users_table_es, 
                          max_steps=1)
result = step_matrix._get_plot_data()[0]
result

,1
catalog,1.0


In [19]:
#assert
result.compare(correct_result).shape == (0, 0)

True

In [20]:
# ещё одна таблица из старых тестов
# если в этих файлах хранятся правильные ответы, то можно использовать и её: 01_basic.csv, 02_one_step.csv, 03_100_steps.csv

source_event123 = pd.DataFrame(
            [
                [1, "event1", "2022-01-01 00:01:00"],
                [1, "event2", "2022-01-01 00:01:02"],
                [1, "event1", "2022-01-01 00:02:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event3", "2022-01-01 00:03:30"],
                [1, "event1", "2022-01-01 00:04:00"],
                [1, "event3", "2022-01-01 00:04:30"],
                [1, "event1", "2022-01-01 00:05:00"],
                [2, "event1", "2022-01-02 00:00:00"],
                [2, "event2", "2022-01-02 00:00:05"],
                [2, "event2", "2022-01-02 00:01:05"],
                [3, "event1", "2022-01-02 00:01:10"],
                [3, "event1", "2022-01-02 00:02:05"],
                [3, "event4", "2022-01-02 00:03:05"],
                [4, "event1", "2022-01-02 00:01:10"],
                [4, "event1", "2022-01-02 00:02:05"],
                [4, "event1", "2022-01-02 00:03:05"],
            ],
            columns=["user_id", "event", "timestamp"],
        )
source_event123

,user_id,event,timestamp
0,1,event1,2022-01-01 00:01:00
1,1,event2,2022-01-01 00:01:02
2,1,event1,2022-01-01 00:02:00
3,1,event1,2022-01-01 00:03:00
4,1,event1,2022-01-01 00:03:00
5,1,event3,2022-01-01 00:03:30
6,1,event1,2022-01-01 00:04:00
7,1,event3,2022-01-01 00:04:30
8,1,event1,2022-01-01 00:05:00
9,2,event1,2022-01-02 00:00:00


In [ ]:
FLOAT_PRECISION = 6 # здесь дроби будут с большим числом знаков после запятой

In [ ]:
def read_test_data(filename):
    # да, файлы с правильными ответами лежат в другой папке
    filepath = os.path.join("../tests/tooling/step_matrix_test_data", filename)
    df = pd.read_csv(filepath, index_col=0).round(FLOAT_PRECISION)
    df.columns = df.columns.astype(int)
    return df

In [ ]:
# всё открывается
# read_test_data('01_basic.csv')

In [ ]:
@pytest.fixture
def stream():
    def remove_start(df, schema):
        return df["event_name"] != "start"

    test_stream = datasets.load_simple_shop()
    graph = PGraph(source_stream=test_stream)
    node1 = EventsNode(StartEndEvents(params=StartEndEventsParams(**{})))
    node2 = EventsNode(FilterEvents(params=FilterEventsParams(filter=remove_start)))

    graph.add_node(node=node1, parents=[graph.root])
    graph.add_node(node=node2, parents=[node1])

    stream = graph.combine(node=node2)
    return stream

In [ ]:
def run_test(stream, filename, **kwargs):
    sm = StepMatrix(eventstream=stream, **kwargs)
    result = sm._get_plot_data()[0]
    result = result.round(FLOAT_PRECISION)
    result_correct = read_test_data(filename)
    test_is_correct = result.compare(result_correct).shape == (0, 0)
    return test_is_correct

In [ ]:
class TestStepMatrix:
    def test_step_matrix_simple(self):
        source_df = pd.DataFrame(
            [
                [1, "event1", "2022-01-01 00:01:00"],
                [1, "event2", "2022-01-01 00:01:02"],
                [1, "event1", "2022-01-01 00:02:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event3", "2022-01-01 00:03:30"],
                [1, "event1", "2022-01-01 00:04:00"],
                [1, "event3", "2022-01-01 00:04:30"],
                [1, "event1", "2022-01-01 00:05:00"],
                [2, "event1", "2022-01-02 00:00:00"],
                [2, "event2", "2022-01-02 00:00:05"],
                [2, "event2", "2022-01-02 00:01:05"],
                [3, "event1", "2022-01-02 00:01:10"],
                [3, "event1", "2022-01-02 00:02:05"],
                [3, "event4", "2022-01-02 00:03:05"],
                [4, "event1", "2022-01-02 00:01:10"],
                [4, "event1", "2022-01-02 00:02:05"],
                [4, "event1", "2022-01-02 00:03:05"],
            ],
            columns=["user_id", "event", "timestamp"],
        )

        correct_result = pd.DataFrame(
            [[1.0, 0.5, 0.5, 0.25, 0.25], [0.0, 0.5, 0.25, 0.0, 0.0], [0.0, 0.0, 0.25, 0.0, 0.0]],
            index=["event1", "event2", "event4"],
            columns=[1, 2, 3, 4, 5],
        )

        source_stream = Eventstream(
            raw_data=source_df,
            raw_data_schema=RawDataSchema(event_name="event", event_timestamp="timestamp", user_id="user_id"),
            schema=EventstreamSchema(),
        )

        sm = StepMatrix(eventstream=source_stream, max_steps=5)
        result= sm._get_plot_data()[0]
        assert result.compare(correct_result).shape == (0, 0)

    def test_step_matrix__basic(self, stream):
        assert run_test(stream, "01_basic.csv")

    def test_step_matrix__one_step(self, stream):
        assert run_test(stream, "02_one_step.csv", max_steps=1)

    def test_step_matrix__100_steps(self, stream):
        assert run_test(stream, "03_100_steps.csv", max_steps=100, precision=3)

In [ ]:
# не запускается(

## Tresholds

In [131]:
source_event123

,user_id,event,timestamp
0,1,event1,2022-01-01 00:01:00
1,1,event2,2022-01-01 00:01:02
2,1,event1,2022-01-01 00:02:00
3,1,event1,2022-01-01 00:03:00
4,1,event1,2022-01-01 00:03:00
5,1,event3,2022-01-01 00:03:30
6,1,event1,2022-01-01 00:04:00
7,1,event3,2022-01-01 00:04:30
8,1,event1,2022-01-01 00:05:00
9,2,event1,2022-01-02 00:00:00


In [22]:
#подготовить данные для теста
data_for_tresh = source_event123
# create data schema
raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

# create source eventstream
source_tresh = Eventstream(
    raw_data=data_for_tresh,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

In [23]:
#let's look at the matrix without using threshold
StepMatrix(eventstream=source_tresh, 
                          max_steps=6)._get_plot_data()[0]

,1,2,3,4,5,6
event1,1.0,0.5,0.50,0.25,0.25,0.00
event2,0.0,0.5,0.25,0.00,0.00,0.00
event4,0.0,0.0,0.25,0.00,0.00,0.00
event3,0.0,0.0,0.00,0.00,0.00,0.25


In [ ]:
# after thresholding the row  THRESHOLDED_X must appear. If we'll choose thresh == 0.3, number of thresholded events will be 4
# event3 appers later then event4, it's important for future correct results dataframe

In [ ]:
# create an insistance of StepMatrix class and save output data
# thresh=0.3
# step_matrix = StepMatrix(eventstream=source_tresh, 
#                           max_steps=6, thresh=0.3)
# result = step_matrix._get_plot_data()[0]
# result
#падает всё с той же ошибкой

In [65]:
correct_result = pd.DataFrame(
            [
                [1.0, 0.5, 0.5, 0.25, 0.25, 0.0],
                [0.0, 0.5, 0.25, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.25, 0.0, 0.0, 0.25]
                
            ], index=['event1', 'event2', 'THRESHOLDED_4'], columns=[1, 2 , 3, 4, 5, 6],
        )
correct_result

,1,2,3,4,5,6
event1,1.0,0.5,0.50,0.25,0.25,0.00
event2,0.0,0.5,0.25,0.00,0.00,0.00
THRESHOLDED_4,0.0,0.0,0.25,0.00,0.00,0.25


In [ ]:
#assert
result.compare(correct_result).shape == (0, 0)

In [ ]:
# create an insistance of StepMatrix class and save output data
# thresh=1.0
#step_matrix = StepMatrix(eventstream=source_tresh, 
#                          max_steps=6, thresh=1.0)
#result = step_matrix._get_plot_data()[0]

In [ ]:
correct_result = pd.DataFrame(
            [
                [1.0, 0.5, 0.5, 0.25, 0.25, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.5, 0.5, 0.0, 0.0, 0.25]
                
            ], index=['event1', 'event2', 'event4', 'event3', 'THRESHOLDED_4'], columns=[1, 2 , 3, 4, 5, 6],
        )
correct_result

!!! Событие попадает в трешхолд, только если оно на всех шагах меньше трешхолда

Таргет и таргет с трешхолдом

## Targets

In [24]:
two_users_table

,user_id,event,timestamp
0,999941967,catalog,2020-04-17 12:18:49.391199
1,999941967,lost,2020-04-17 12:18:50.391199
2,530349536,catalog,2019-11-12 10:03:06.441295
3,530349536,cart,2019-11-12 10:03:34.490371
4,530349536,delivery_choice,2019-11-12 10:03:36.538909
5,530349536,delivery_courier,2019-11-12 10:03:38.013087
6,530349536,payment_choice,2019-11-12 10:03:39.435086
7,530349536,payment_card,2019-11-12 10:03:39.650894


In [28]:
# create source eventstream
two_users_table = Eventstream(
    raw_data=two_users_table,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

In [85]:
# without threshhold
step_matrix = StepMatrix(eventstream=two_users_table_es, 
                          max_steps=6)
result = step_matrix._get_plot_data()[0]
display(result)
result.index

,1,2,3,4,5,6
catalog,1.0,0.0,0.0,0.0,0.0,0.0
cart,0.0,0.5,0.0,0.0,0.0,0.0
delivery_choice,0.0,0.0,0.5,0.0,0.0,0.0
delivery_courier,0.0,0.0,0.0,0.5,0.0,0.0
payment_choice,0.0,0.0,0.0,0.0,0.5,0.0
payment_card,0.0,0.0,0.0,0.0,0.0,0.5
lost,0.0,0.5,0.0,0.0,0.0,0.0


Index(['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'lost'],
      dtype='object')

In [55]:
# without threshhold
correct_result = pd.DataFrame(
            [
                [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.5, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.5, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.5, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.5, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.5],
                [0.0, 0.5, 0.0, 0.0, 0.0, 0.0]
                
            ], index=(['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'lost']), columns=[1, 2 , 3, 4, 5, 6],
        )
correct_result

,1,2,3,4,5,6
catalog,1.0,0.0,0.0,0.0,0.0,0.0
cart,0.0,0.5,0.0,0.0,0.0,0.0
delivery_choice,0.0,0.0,0.5,0.0,0.0,0.0
delivery_courier,0.0,0.0,0.0,0.5,0.0,0.0
payment_choice,0.0,0.0,0.0,0.0,0.5,0.0
payment_card,0.0,0.0,0.0,0.0,0.0,0.5
lost,0.0,0.5,0.0,0.0,0.0,0.0


Посмотреть где меняется порядок строчек, в таблице или на графике

In [43]:
# with threshhold = 0.05
#новая тестовая таблица
tree_users_table = two_users_table.append((raw_data.loc[raw_data['user_id'] == 760510988]).sample(frac=0.05, random_state=1)).reset_index(drop=True)
four_users_table = tree_users_table.append((raw_data.loc[raw_data['user_id'] == 78179456]).sample(frac=0.05, random_state=1)).reset_index(drop=True)
four_users_table
#tree_users_table

,user_id,event,timestamp
0,999941967,catalog,2020-04-17 12:18:49.391199
1,999941967,lost,2020-04-17 12:18:50.391199
2,530349536,catalog,2019-11-12 10:03:06.441295
3,530349536,cart,2019-11-12 10:03:34.490371
4,530349536,delivery_choice,2019-11-12 10:03:36.538909
5,530349536,delivery_courier,2019-11-12 10:03:38.013087
6,530349536,payment_choice,2019-11-12 10:03:39.435086
7,530349536,payment_card,2019-11-12 10:03:39.650894
8,760510988,main,2019-11-21 16:19:55.114650
9,760510988,cart,2020-02-28 07:59:40.094463


In [44]:
# create source eventstream
four_users_table_es = Eventstream(
    raw_data=four_users_table,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

In [58]:
# with threshhold
#step_matrix = StepMatrix(eventstream=four_users_table_es, 
 #                         max_steps=6, targets=['payment_card', 'payment_cash'], thresh=0.5)
result = step_matrix._get_plot_data()[0]
display(result)

print(result.index)

,1,2,3,4,5,6
catalog,0.75,0.50,0.50,0.00,0.00,0.00
cart,0.00,0.25,0.00,0.25,0.00,0.00
delivery_choice,0.00,0.00,0.25,0.00,0.00,0.00
delivery_courier,0.00,0.00,0.00,0.25,0.00,0.00
payment_choice,0.00,0.00,0.00,0.00,0.25,0.00
payment_card,0.00,0.00,0.00,0.00,0.00,0.25
lost,0.00,0.25,0.00,0.00,0.00,0.00
payment_cash,0.00,0.00,0.00,0.25,0.00,0.00
main,0.25,0.00,0.00,0.00,0.00,0.00


Index(['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'lost', 'payment_cash', 'main'],
      dtype='object')


In [64]:
# correct_result with teshhold = 0.5
correct_result = pd.DataFrame(
            [
                [0.75, 0.5, 0.5, 0.0, 0.0, 0.0],
                [0.25, 0.5, 0.25, 0.5, 0.25, 0.0], #THRESHOLDED_6
                [0.0, 0.0, 0.0, 0.25, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.25]
                
            ], index=(['catalog','THRESHOLDED_6',  'payment_card',  'payment_cash']), 
                columns=[1, 2 , 3, 4, 5, 6]
        )
correct_result

,1,2,3,4,5,6
catalog,0.75,0.5,0.50,0.00,0.00,0.00
THRESHOLDED_6,0.25,0.5,0.25,0.50,0.25,0.00
payment_card,0.00,0.0,0.00,0.25,0.00,0.00
payment_cash,0.00,0.0,0.00,0.00,0.00,0.25


In [ ]:
#grouping targets
#step_matrix = StepMatrix(eventstream=four_users_table_es, 
 #                         max_steps=6, targets=[['payment_card', 'payment_cash']], thresh=0.5)
result = step_matrix._get_plot_data()[0]


In [ ]:
# correct_result with teshhold = 0.5 and grouping targets

correct_result = pd.DataFrame(
            [
                [0.75, 0.5, 0.5, 0.0, 0.0, 0.0],
                [0.25, 0.5, 0.25, 0.5, 0.25, 0.0], #THRESHOLDED_6
                [0.0, 0.0, 0.0, 0.25, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.25]
                
            ], index=(['catalog','THRESHOLDED_6',  'payment_card',  'payment_cash']), 
                columns=[1, 2 , 3, 4, 5, 6]
        )
correct_result

## Аккумулирование 

In [86]:
#raw_data.query('event == "payment_card" ')

In [79]:
card_payers_table = two_users_table.append([raw_data.loc[raw_data['user_id'] == 831095347], raw_data.loc[raw_data['user_id'] == 962761227],  raw_data.loc[raw_data['user_id'] == 343679414] ]).reset_index(drop=True)
card_payers_table = card_payers_table.loc[card_payers_table['user_id'] != 999941967]
card_payers_table #4 пользователя с оплатой картой

,user_id,event,timestamp
2,530349536,catalog,2019-11-12 10:03:06.441295
3,530349536,cart,2019-11-12 10:03:34.490371
4,530349536,delivery_choice,2019-11-12 10:03:36.538909
5,530349536,delivery_courier,2019-11-12 10:03:38.013087
6,530349536,payment_choice,2019-11-12 10:03:39.435086
7,530349536,payment_card,2019-11-12 10:03:39.650894
8,831095347,catalog,2019-11-08 04:36:40.677305
9,831095347,product2,2019-11-08 04:36:51.471034
10,831095347,catalog,2019-11-08 04:36:55.620954
11,831095347,catalog,2019-11-08 04:37:09.559781


In [81]:
# create source eventstream
card_payers_table_es = Eventstream(
    raw_data=card_payers_table,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

In [92]:
step_matrix = StepMatrix(eventstream=card_payers_table_es, 
                      max_steps=11, targets=['payment_card'])
result = step_matrix._get_plot_data()[0]
display(result)

print(result.index)

,1,2,3,4,5,6,7,8,9,10,11
catalog,0.75,0.50,0.50,0.50,0.00,0.50,0.00,0.00,0.25,0.25,0.00
cart,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,0.00,0.00,0.00
delivery_choice,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.00,0.00,0.00
delivery_courier,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.00,0.00
payment_choice,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.00
payment_card,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25
main,0.25,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.00,0.00,0.25
payment_done,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25
product1,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00
product2,0.00,0.25,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00


Index(['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'main', 'payment_done', 'product1',
       'product2'],
      dtype='object')


In [ ]:
#step_matrix = StepMatrix(eventstream=card_payers_table_es, 
 #                         max_steps=11, 
                         # targets=['payment_card'], 
        #                  thresh=0.5,
        #                  accumulated='only' )
result = step_matrix._get_plot_data()[0]
display(result)

print(result.index)

In [94]:
# accumulated='only'
# with teshhold = 0.5
correct_result = pd.DataFrame(
            [
                [0.75, 0.5, 0.5, 0.5, 0.0, 0.5, 0.00, 0.0, 0.25, 0.25, 0.00],
                [0.25, 0.5, 0.5, 0.5, 1.0, 0.25, 0.75, 0.75, 0.5, 0.25, 0.5], #THRESHOLDED_8
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.5, 0.75], #payment_card_acc

            ], index=(['catalog','THRESHOLDED_8', 'ACC_payment_card']), 
                columns=[1, 2 , 3, 4, 5, 6, 7, 8, 9, 10, 11]
        )
correct_result

,1,2,3,4,5,6,7,8,9,10,11
catalog,0.75,0.5,0.5,0.5,0.0,0.50,0.00,0.00,0.25,0.25,0.00
THRESHOLDED_8,0.25,0.5,0.5,0.5,1.0,0.25,0.75,0.75,0.50,0.25,0.50
ACC_payment_card,0.00,0.0,0.0,0.0,0.0,0.25,0.25,0.25,0.25,0.50,0.75


In [95]:
# accumulated='both'
# with teshhold = 0.5
correct_result = pd.DataFrame(
            [
                [0.75, 0.5, 0.5, 0.5, 0.0, 0.5, 0.00, 0.0, 0.25, 0.25, 0.00],
                [0.25, 0.5, 0.5, 0.5, 1.0, 0.25, 0.75, 0.75, 0.5, 0.25, 0.5], #THRESHOLDED_8
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.25, 0.25], #payment_card
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.5, 0.75], #payment_card_acc

            ], index=(['catalog','THRESHOLDED_8', 'payment_card', 'ACC_payment_card']), 
                columns=[1, 2 , 3, 4, 5, 6, 7, 8, 9, 10, 11]
        )
correct_result

,1,2,3,4,5,6,7,8,9,10,11
catalog,0.75,0.5,0.5,0.5,0.0,0.50,0.00,0.00,0.25,0.25,0.00
THRESHOLDED_8,0.25,0.5,0.5,0.5,1.0,0.25,0.75,0.75,0.50,0.25,0.50
payment_card,0.00,0.0,0.0,0.0,0.0,0.25,0.00,0.00,0.00,0.25,0.25
ACC_payment_card,0.00,0.0,0.0,0.0,0.0,0.25,0.25,0.25,0.25,0.50,0.75


In [ ]:
# центрирование (с таргетом и без)

In [100]:
card_payers_table = card_payers_table.query("event != 'product1' and event != 'product2' and event != 'lost'")
step_matrix._get_plot_data()[0]

In [105]:
card_payers_table

,user_id,event,timestamp
2,530349536,catalog,2019-11-12 10:03:06.441295
3,530349536,cart,2019-11-12 10:03:34.490371
4,530349536,delivery_choice,2019-11-12 10:03:36.538909
5,530349536,delivery_courier,2019-11-12 10:03:38.013087
6,530349536,payment_choice,2019-11-12 10:03:39.435086
7,530349536,payment_card,2019-11-12 10:03:39.650894
8,831095347,catalog,2019-11-08 04:36:40.677305
10,831095347,catalog,2019-11-08 04:36:55.620954
11,831095347,catalog,2019-11-08 04:37:09.559781
13,831095347,catalog,2019-11-08 04:37:38.516478


In [102]:
# create source eventstream
card_payers_table_es = Eventstream(
    raw_data=card_payers_table,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

In [107]:
step_matrix = StepMatrix(eventstream=card_payers_table_es, 
                          max_steps=11, 
                          centered={'event':'payment_card',
                                  'left_gap':5,
                                  'occurrence':1})
result = step_matrix._get_plot_data()[0]
display(result)

print(result.index)

,1,2,3,4,5,6,7,8,9,10,11
catalog,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
cart,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
delivery_choice,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0
delivery_courier,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0
payment_choice,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0
payment_card,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0
payment_done,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0,0,0,0


Index(['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'payment_done'],
      dtype='object')


In [108]:
correct_result = pd.DataFrame(
            [
                [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], #payment_card
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0],

            ], index=(['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'payment_done']), 
                columns=[1, 2 , 3, 4, 5, 6, 7, 8, 9, 10, 11]
        )
correct_result

,1,2,3,4,5,6,7,8,9,10,11
catalog,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cart,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
delivery_choice,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
delivery_courier,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
payment_choice,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
payment_card,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
payment_done,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0


In [ ]:
# центрирование (с таргетом)

In [111]:
step_matrix = StepMatrix(eventstream=card_payers_table_es, 
                          max_steps=11, 
                          centered={'event':'payment_card',
                                  'left_gap':5,
                                  'occurrence':1},
                          targets=['payment_done'],
                          thresh = 0.6)
#result = step_matrix._get_plot_data()[0]


In [112]:
correct_result = pd.DataFrame(
            [
                [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], #payment_card
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], #THRESHOLDED_0
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0], #payment_done

            ], index=(['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'THRESHOLDED_0', 'payment_done']), 
                columns=[1, 2 , 3, 4, 5, 6, 7, 8, 9, 10, 11]
        )
correct_result

,1,2,3,4,5,6,7,8,9,10,11
catalog,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cart,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
delivery_choice,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
delivery_courier,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
payment_choice,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
payment_card,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
THRESHOLDED_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
payment_done,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0


In [121]:
step_matrix = StepMatrix(eventstream=card_payers_table_es, 
                          max_steps=6)
step_matrix._get_plot_data()[0]

,1,2,3,4,5,6
catalog,0.75,0.75,0.50,0.50,0.25,0.00
cart,0.00,0.25,0.00,0.25,0.50,0.00
delivery_choice,0.00,0.00,0.25,0.00,0.00,0.50
delivery_courier,0.00,0.00,0.00,0.25,0.00,0.00
payment_choice,0.00,0.00,0.00,0.00,0.25,0.00
main,0.25,0.00,0.25,0.00,0.00,0.25
payment_card,0.00,0.00,0.00,0.00,0.00,0.25


In [123]:
# Custom events sorting
new_order = ['payment_card', 'main', 'catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice']


step_matrix = StepMatrix(eventstream=card_payers_table_es, 
                          max_steps=6)

step_matrix_new_order = StepMatrix(eventstream=card_payers_table_es, 
                                   sorting=new_order, 
                                   max_steps=6)
result = step_matrix._get_plot_data()[0], step_matrix_new_order._get_plot_data()[0]
display(result)

print(result.index)

(                     1     2     3     4     5     6
 catalog           0.75  0.75  0.50  0.50  0.25  0.00
 cart              0.00  0.25  0.00  0.25  0.50  0.00
 delivery_choice   0.00  0.00  0.25  0.00  0.00  0.50
 delivery_courier  0.00  0.00  0.00  0.25  0.00  0.00
 payment_choice    0.00  0.00  0.00  0.00  0.25  0.00
 main              0.25  0.00  0.25  0.00  0.00  0.25
 payment_card      0.00  0.00  0.00  0.00  0.00  0.25,
                      1     2     3     4     5     6
 payment_card      0.00  0.00  0.00  0.00  0.00  0.25
 main              0.25  0.00  0.25  0.00  0.00  0.25
 catalog           0.75  0.75  0.50  0.50  0.25  0.00
 cart              0.00  0.25  0.00  0.25  0.50  0.00
 delivery_choice   0.00  0.00  0.25  0.00  0.00  0.50
 delivery_courier  0.00  0.00  0.00  0.25  0.00  0.00
 payment_choice    0.00  0.00  0.00  0.00  0.25  0.00)

<built-in method index of tuple object at 0x7f914b2835c0>


In [129]:
# Custom events sorting
correct_result = (pd.DataFrame(
            [
                [0.75, 0.75, 0.50, 0.50, 0.25, 0.0],  #catalog
                [0.0, 0.25, 0.0, 0.25, 0.50, 0.0],
                [0.0, 0.0, 0.25, 0.0, 0.0, 0.5],
                [0.0, 0.0, 0.0, 0.25, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.25, 0.0],
                [0.25, 0.0, 0.25, 0.0, 0.0, 0.25], # main
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.25]    #payment_card                    #old order
            ], index=(['catalog', 'cart', 'delivery_choice', 'delivery_courier',     #old order
       'payment_choice', 'main', 'payment_card']), 
                columns=[1, 2 , 3, 4, 5, 6]), 
pd.DataFrame(
            [
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.25], #payment_card
                [0.25, 0.0, 0.25, 0.0, 0.0, 0.25],  # main
                [0.75, 0.75, 0.50, 0.50, 0.25, 0.0],   #catalog
                [0.0, 0.25, 0.0, 0.25, 0.50, 0.0],
                [0.0, 0.0, 0.25, 0.0, 0.0, 0.5],
                [0.0, 0.0, 0.0, 0.25, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.25, 0.0],
                
                 #new order

            ], index=(['payment_card',  'main', 'catalog', 'cart', 'delivery_choice', 'delivery_courier',     #old order
       'payment_choice']), 
                columns=[1, 2 , 3, 4, 5, 6],
        ))
correct_result

(                     1     2     3     4     5     6
 catalog           0.75  0.75  0.50  0.50  0.25  0.00
 cart              0.00  0.25  0.00  0.25  0.50  0.00
 delivery_choice   0.00  0.00  0.25  0.00  0.00  0.50
 delivery_courier  0.00  0.00  0.00  0.25  0.00  0.00
 payment_choice    0.00  0.00  0.00  0.00  0.25  0.00
 main              0.25  0.00  0.25  0.00  0.00  0.25
 payment_card      0.00  0.00  0.00  0.00  0.00  0.25,
                      1     2     3     4     5     6
 payment_card      0.00  0.00  0.00  0.00  0.00  0.25
 main              0.25  0.00  0.25  0.00  0.00  0.25
 catalog           0.75  0.75  0.50  0.50  0.25  0.00
 cart              0.00  0.25  0.00  0.25  0.50  0.00
 delivery_choice   0.00  0.00  0.25  0.00  0.00  0.50
 delivery_courier  0.00  0.00  0.00  0.25  0.00  0.00
 payment_choice    0.00  0.00  0.00  0.00  0.25  0.00)

## Differential step matrix

In [ ]:
#dataset preparing
# 4 users, 2 groups


In [ ]:
# Differential step matrix
g1 = set(data[data['event']=='payment_done']['user_id'])
g2 = set(data['user_id']) - g1

data.rete.step_matrix(max_steps=16,
                      thresh = 0.05,
                      centered={'event':'cart',
                                'left_gap':5,
                                'occurrence':1},
                      groups=(g1, g2));

In [133]:
diff_matr_table_p1 = card_payers_table.query('user_id == "530349536" or user_id == "831095347"' )
diff_matr_table = diff_matr_table_p1.append('')